# Тестовое задание Павла Спирина

## <a name="0.0"></a>Содержание:
* [Описание](#0.)
* [Задание 1](#1.1.)
    - [Часть 1 и 2](#1.1.1.)
    - [Часть 3 - Средняя зарплата по каждой специальности](#1.1.2.)
    - [Часть 3 - Самые встречаемые би-граммы по каждой специальности](#1.1.3.) 
* [Задание 2](#2.1.)
* [Задание 3](#3.1.)

<a name="0."></a>
## Описание
<font size="2">([к содержанию](#0.0))</font>

Задание должно быть выполнено в виртуальной среде Google Colab, должна быть предоставлена ссылка на рабочий ноутбук. Все ячейки должны запускаться последовательно без вылетов. Если какая-то ячейка не запускается задание далее проверяться не будет. В ноутбуке должны быть предоставлены комментарии по работе кода.  
**Задание 1**

1. Необходимо создать код по закачке вакансий с hh.ru через API. Итоговый датафрейм должен содержать основную информацию по вакансиям: название, уровень опыта, ключевые навыки, профессиональную роль (ОБЯЗОНОСТИ или ФУНКЦИОНАЛ), ссылку на вакансию, город, зарплату.  
2. Закачать вакансии по специальностям: разработчик Java, SMM, интернет-маркетолог и веб-дизайнер.  
3. Показать: среднюю ЗП по каждой из специальностей и 10 самых частых би-грамм для каждой из специальностей

**Задание 2**  
Возьмите два файла с вакансиями, прилагающиеся к тексту задания (internet_marketologs_part_1.xlsx и internet_marketologs_part_2.xlsx.) Создайте датафрейм с вакансиями, вошедшими в первую закачку, но не вошедшими во вторую.

**Задание 3**  
Постройте прототип рекомендательной системы на основе данных, полученных из закачек в задании 1. Система должна выдавать нужную вакансию при запросе. Например, мы даем ей ключевые навыки или текст резюме интернет маркетолога, на что система выдает нам 10 подходящих вакансий из закачанной в задании 1 базе.

**Срок выполнения: 7 марта 2024**

<a name="1.1."></a>
## Задание 1
<font size="2">([к содержанию](#0.0))</font>


<a name="1.1.1."></a>
### Часть 1 и 2
<font size="2">([к содержанию](#0.0))</font>

Библиотеки буду импортировать отдельно для каждого блока заданий

In [1]:
import pandas as pd
import requests

Определим три функции:
- `get_vacancies(keyword, per_page=100, page=0)`: Получает список вакансий по ключевому слову через API и возвращает его в формате JSON.
- `get_vacancy_details(vacancy_id)`: Получает дополнительные детали вакансии по ее ID через API и возвращает их в формате JSON.
- `extract_info(vacancy, specialization)`: Извлекает основную информацию о вакансии из JSON-объекта и возвращает ее в виде словаря.

`get_vacancies`: Эта функция отправляет запрос к API HeadHunter с использованием ключевого слова поиска и параметров количества вакансий на странице и номера страницы. Она возвращает список вакансий (в формате JSON), полученных в ответ на запрос.

In [2]:
def get_vacancies(keyword, per_page=100, page=0):
    """
    - Получает вакансии по ключевому слову через API;
    -Параметры:
        - keyword: str
            Ключевое слово для поиска вакансий.
        - per_page: int, optional (default=100)
            Количество вакансий, возвращаемых на одной странице.
        - page: int, optional (default=0)
            Номер страницы результатов, начиная с нуля.
    - Возвращает список вакансий в формате JSON.
    
    """
    url = f'https://api.hh.ru/vacancies?text={keyword}&per_page={per_page}&page={page}'
    response = requests.get(url)
    data = response.json()
    vacancies = data['items']
    return vacancies

`get_vacancy_details`: Эта функция принимает идентификатор вакансии и отправляет запрос к API HeadHunter для получения дополнительных деталей о конкретной вакансии (например, зарплата, опыт работы, ключевые навыки и т. д.). Она возвращает информацию о вакансии в формате JSON.

In [3]:
def get_vacancy_details(vacancy_id):
    """
    - Получает дополнительные детали вакансии по ее ID через API;
    - Параметры:
        - vacancy_id: строка, идентификатор вакансии;
    - Возвращает детали вакансии в формате JSON.

    """
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    response = requests.get(url)
    data = response.json()
    return data

`extract_info`: Эта функция принимает информацию о вакансии в формате JSON, извлекает из нее основные данные, такие как название вакансии, зарплата, опыт работы, навыки, специализация, описание вакансии, город и ссылка на вакансию. Она формирует словарь с этими данными и возвращает его.

In [4]:
def extract_info(vacancy, specialization):
    """
    Извлекает основную информацию о вакансии из JSON-объекта.

    Параметры:
    - vacancy: JSON-объект, содержащий информацию о вакансии;
    - specialization: строка, специализация, для которой извлекается информация о вакансии.

    Возвращает словарь с основной информацией о вакансии.

    """
    
    # Извлекаем основную информацию о вакансии
    title = vacancy.get('name')
    
    salary = vacancy.get('salary')
    
    experience = vacancy.get('experience', {}).get('name')
    
    skills = [skill['name'] for skill in vacancy.get('key_skills', [])]
    
    specialization_mapping = {
        'разработчик Java': 'Java Developer',
        'SMM': 'SMM',
        'интернет-маркетолог': 'Internet Marketer',
        'веб-дизайнер': 'Web Designer'
    }
    specialization = specialization_mapping.get(specialization)

    prof_specialization = vacancy.get('specializations', [])
    professional_role = prof_specialization[0].get('profarea_name') if prof_specialization else None
    
    duties = vacancy.get('description')
    
    city = vacancy.get('area', {}).get('name')
    
    link = vacancy.get('alternate_url')
    
    return {
        'Title': title,
        'Salary': salary,
        'Experience': experience,
        'Skills': skills,
        'Professional Role': professional_role,
        'Specialization': specialization,
        'Duties': duties,
        'City': city,
        'Link': link
    }

Функция main является основной функцией. В ней мы последовательно вызываем остальные функций для выполнения задачи парсинга

Действия функции main:

- Определяет список специализаций specialties, для которых нужно получить вакансии.
- Создает пустой список data для хранения данных о вакансиях.
- Получает вакансии для каждой специализации с помощью функции get_vacancies и добавляет их в список all_vacancies.
- Получает дополнительные детали о каждой вакансии с помощью функции get_vacancy_details и сохраняет их в список vacancy_details.
- Извлекает нужную информацию о вакансиях из полученных данных с помощью функции extract_info и добавляет ее в список data.
- Создает DataFrame из списка данных о вакансиях.
- Возвращает этот DataFrame.

In [5]:
# Создание пустого список для хранения данных
data = []

Определим основную функцию `main()`

In [6]:
def main():
    """
    - Основная функция программы.
    - Собирает данные о вакансиях по заданным специализациям.
    - Извлекает информацию о вакансиях и заполняет список данными.
    - Создает DataFrame из списка данных.

    """
    # Специализации для поиска вакансий
    specialties = ['разработчик Java', 'SMM', 'интернет-маркетолог', 'веб-дизайнер']
    
    # Создаем пустой список для хранения данных
    data = []

    # Собираем данные о вакансиях
    all_vacancies = []
    for specialty in specialties:
        vacancies = get_vacancies(specialty)
        all_vacancies.extend(vacancies)
    
    # Извлекаем информацию о вакансиях
    vacancy_details = []
    for vacancy in all_vacancies:
        details = get_vacancy_details(vacancy['id'])
        vacancy_details.append(details)
    
    # Заполняем список данными о вакансиях
    for i, vacancy in enumerate(vacancy_details):
        prof_specialization = specialties[i % len(specialties)]  # Циклический выбор специализации
        data.append(extract_info(vacancy, prof_specialization))
        
    
    # Создаем DataFrame из списка данных
    df = pd.DataFrame(data)
    
    return df

In [7]:
# Запуск основной функции по парсингу данных
df = main()

In [8]:
# df.to_csv('vacancies_dataset.csv', index=False) # Сохранение датасета если нужно

Признак `Professional Role` видимо не заполняется работодателем (так и не смог вытащить из него информацию).

Так же я добавил признак `Specialization`, в который добавляется название специализации при парсинге.

In [9]:
df.count()

Title                146
Salary                89
Experience           146
Skills               400
Professional Role      0
Specialization       400
Duties               146
City                 146
Link                 146
dtype: int64

In [10]:
df.head()

,Title,Salary,Experience,Skills,Professional Role,Specialization,Duties,City,Link
0,Frontend Developer (React),None,От 1 года до 3 лет,"[JavaScript, React, TypeScript, MobX, Bootstra...",None,Java Developer,"<p>Мы аккредитованная IT-компания, основанная ...",Екатеринбург,https://hh.ru/vacancy/94287939
1,HTML-верстальщик (HTML / CSS / JavaScript),"{'from': 70000, 'to': 100000, 'currency': 'RUR...",От 1 года до 3 лет,"[ReactJS, HTML, CSS, HTML5, JavaScript, Git, C...",None,SMM,<p>Привет!</p> <p>Мы - компания ISS.Digital. М...,Москва,https://hh.ru/vacancy/94318962
2,Java программист,None,От 1 года до 3 лет,"[Java, PostgreSQL, Spring Boot, ORACLE]",None,Internet Marketer,<p><strong>ПРОЕКТ:</strong></p> <p>Ищем разраб...,Воронеж,https://hh.ru/vacancy/94268107
3,Ведущий программист Java (удалённо),None,От 3 до 6 лет,"[DevOps (Git (bitbucket), Jenkins, ansible, ma...",None,Web Designer,<p><em><strong>КОМПАНИЯ ООО ИЦ «АЙ-ТЕКО»</stro...,Москва,https://hh.ru/vacancy/92430462
4,Младший разработчик,"{'from': 60000, 'to': None, 'currency': 'RUR',...",Нет опыта,[],None,Java Developer,<p>Вакансия для начинающих разработчиков с неб...,Санкт-Петербург,https://hh.ru/vacancy/94254962


Так как признак `Professional Role` всегда None, я позволил себе добавить признак `Duties`, в котором мы видим как и обязоности (по сути профессиональная роль), так описание компании и требования к кандидату. Это сделал для того, чтобы в дальнейшем была хоть какая-то информация для выявления 10 самых частых би-грамм для каждой из специальностей.

Так же добавил признак `Specialization` для того, чтобы в дальнешем построить сводную таблицу по средним зарплатам

In [11]:
df.Duties[0]

'<p>Мы аккредитованная IT-компания, основанная в 2015 году<strong> </strong>как одно из R&amp;D-подразделений группы компаний <strong>S&amp;T Iskratel</strong> (дочерняя структура группы <strong>Kontron AG</strong>, Австрия). Основные направления нашей деятельности - это продукты и решения по цифровизации и автоматизации производственных и управленческих процессов в таких отраслях как транспорт, энергетика, общественная безопасность и телекоммуникации.</p> <p>Одним из новых направлений работы для нас является проект по разработке ПО ядра современных сетей связи на базе технологий 4G/5G. Данный проект предполагает реализацию решения, обеспечивающего необходимые функции мобильных сетей четвертого и пятого поколения в соответствии со спецификациями комитета по стандартизации технологий мобильной связи 3GPP. Большой объем работы по этим проектам предполагает расширение команды, поэтому мы <strong>ищем опытного Frontend разработчика (React)</strong>.</p> <p><strong>Наш стек: </strong>Typesc

In [12]:
# Удаление вакансий с отсутствующей ссылкой
df = df.dropna(subset=['Link'])

In [13]:
df.isnull().sum()

Title                  0
Salary                57
Experience             0
Skills                 0
Professional Role    146
Specialization         0
Duties                 0
City                   0
Link                   0
dtype: int64

Датасет df готов к дальнейшей работе с данными

<a name="1.1.2."></a>
### Часть 3 - Средняя зарплата по каждой специальности
<font size="2">([к содержанию](#0.0))</font>

In [14]:
import pandas as pd

In [15]:
# Создание копии DataFrame
analytical_df = df.copy()

In [16]:
analytical_df.Salary

0                                                   None
1      {'from': 70000, 'to': 100000, 'currency': 'RUR...
2                                                   None
3                                                   None
4      {'from': 60000, 'to': None, 'currency': 'RUR',...
                             ...                        
395    {'from': 40000, 'to': None, 'currency': 'RUR',...
396                                                 None
397    {'from': None, 'to': 40000, 'currency': 'RUR',...
398    {'from': 100000, 'to': 150000, 'currency': 'KZ...
399    {'from': 75000, 'to': None, 'currency': 'RUR',...
Name: Salary, Length: 146, dtype: object

In [17]:
analytical_df.City.unique()

array(['Екатеринбург', 'Москва', 'Воронеж', 'Санкт-Петербург', 'Алматы',
       'Минск', 'Красноярск', 'Лида', 'Пермь', 'Ташкент', 'Таганрог',
       'Казань', 'Тольятти', 'Нижний Новгород', 'Якутск', 'Барнаул',
       'Новосибирск', 'Брест', 'Астана', 'Кемерово', 'Сербия', 'Заславль',
       'Волгоград', 'Краснодар', 'Тбилиси', 'Челябинск', 'Петрозаводск',
       'Омск'], dtype=object)

Созадим две функции:  

`convert_to_rub()` - Функция по конвертации валют в рубли по следующим курсам:
- 1 RUR = 1₽
- 1 USD = 90₽
- 1 EUR = 100₽
- 1 KZT = 0,20₽
- 1 BYR = 28₽
- 1 KGS = 1₽
- 1 UZS = 0,0073₽

`extract_salary_info()` - Функция выделяет новые признаки `From salary` и `To salary` из основного признака `Salary`. Эти признаки будут отражать нижний и верхний предел зарплаты. А так же конвертирует валюту в рубли

Так как валют много, то буду вносить значения курсов обмена по умолчанию внутри самой функции, а не в аргументе функции.

In [18]:
def convert_to_rub(amount, currency):
    """
    Конвертирует сумму зарплаты в валюте в рубли.

    Параметры:
        - amount (float): Сумма зарплаты;
        - currency (str): Валюта зарплаты ('USD', 'EUR', 'KZT', 'BYR', 'KGS').
    
    Возвращает:
        float: Сумма зарплаты, сконвертированная в рубли, или None, если сумма равна None.
        
    """
    if amount is None:
        return None
    
    # Установка курса обмена по умолчанию для неизвестных валют
    exchange_rate = {
        'RUR': 1,
        'USD': 90,
        'EUR': 100,
        'KZT': 0.20,
        'BYR': 28,
        'KGS': 1,
        'UZS': 0.0073
    }


    # Получаем курс обмена для заданной валюты, если валюта не найдена возвращаем None
    rate = exchange_rate.get(currency)
    if rate is None:
        return None
    
    # Возвращаем сконвертированную сумму
    return amount * rate

In [19]:
def extract_salary_info(salary_info):
    """
    Извлекает информацию о зарплате из словаря с данными о зарплате.

    Параметры:
        - salary_info (dict): Словарь с данными о зарплате.

    Возвращает:
        tuple: Кортеж из двух элементов:
            - from_salary (float): Сумма начальной зарплаты, сконвертированная в рубли, или None, если отсутствует.
            - to_salary (float): Сумма конечной зарплаты, сконвертированная в рубли, или None, если отсутствует.
    """
    if salary_info:
        from_salary = salary_info.get('from')
        to_salary = salary_info.get('to')
        currency = salary_info.get('currency')
        from_salary_rub = convert_to_rub(from_salary, currency)
        to_salary_rub = convert_to_rub(to_salary, currency)
        return from_salary_rub, to_salary_rub
    else:
        return None, None

In [20]:
# Применяем функцию extract_salary_info к столбцу Salary для создания двух новых столбцов
analytical_df[['From Salary', 'To Salary']] = analytical_df['Salary'].apply(lambda x: pd.Series(extract_salary_info(x)))

In [21]:
analytical_df.loc[:5, ['Salary', 'From Salary', 'To Salary']]

,Salary,From Salary,To Salary
0,None,NaN,NaN
1,"{'from': 70000, 'to': 100000, 'currency': 'RUR...",70000.0,100000.0
2,None,NaN,NaN
3,None,NaN,NaN
4,"{'from': 60000, 'to': None, 'currency': 'RUR',...",60000.0,NaN
5,"{'from': 120000, 'to': 120000, 'currency': 'RU...",120000.0,120000.0


Далее расчитаем среднюю зарплату для каждой вакансии по условию:  
- Если в вакансии есть только один признак `From Salary` или  `To Salary`, то он будет отражать среднюю зарплату;
- Если признаки `From Salary` и  `To Salary` равен числу, то посчитаем среднюю зарплату

Первое условие позволит не потерять информацию из тех вакансий, где заполнено только одно значение. 

Добавим среднюю зарплату в новые признак `Mean Salary`

In [22]:
def calculate_average_salary(row):
    """
    Рассчитывает среднюю зарплату для каждой вакансии на основе признаков 'From Salary' и 'To Salary'.

    Параметры:
        - row (pd.Series): Строка (набор признаков) из DataFrame.

    Возвращает:
        float: Средняя зарплата для вакансии.
    """
    from_salary = row['From Salary']
    to_salary = row['To Salary']
    
    # Если один из признаков отсутствует, средняя зарплата будет равна существующему значению
    if pd.notnull(from_salary) and pd.isnull(to_salary):
        return from_salary
    elif pd.isnull(from_salary) and pd.notnull(to_salary):
        return to_salary
    
    # Если оба признака существуют, рассчитываем среднюю зарплату
    elif pd.notnull(from_salary) and pd.notnull(to_salary):
        return (from_salary + to_salary) / 2
    
    # Если оба признака отсутствуют, возвращаем None
    else:
        return None

In [23]:
# Создаем новый столбец 'Mean Salary' и применяем функцию calculate_average_salary к каждой строке DataFrame
analytical_df['Mean Salary'] = analytical_df.apply(calculate_average_salary, axis=1)

In [24]:
# Выведем первые 10 вакансий с нужными признаками
analytical_df.loc[:, ['Salary', 'From Salary', 'To Salary', 'Mean Salary']].head(10)

,Salary,From Salary,To Salary,Mean Salary
0,None,NaN,NaN,NaN
1,"{'from': 70000, 'to': 100000, 'currency': 'RUR...",70000.0,100000.0,85000.0
2,None,NaN,NaN,NaN
3,None,NaN,NaN,NaN
4,"{'from': 60000, 'to': None, 'currency': 'RUR',...",60000.0,NaN,60000.0
5,"{'from': 120000, 'to': 120000, 'currency': 'RU...",120000.0,120000.0,120000.0
6,"{'from': None, 'to': 500000, 'currency': 'KZT'...",NaN,100000.0,100000.0
7,"{'from': 80000, 'to': None, 'currency': 'RUR',...",80000.0,NaN,80000.0
8,None,NaN,NaN,NaN
9,None,NaN,NaN,NaN


Посчитаем среднюю зарплату для каждой специальности

In [25]:
# Создание сводной таблицы
salary_pivot_table = analytical_df.pivot_table(index='Specialization', values='Mean Salary', aggfunc='mean').round(2)

In [26]:
salary_pivot_table.sort_values(by='Mean Salary', ascending=False)

,Mean Salary
Specialization,
SMM,127658.33
Java Developer,118425.00
Web Designer,95340.91
Internet Marketer,92913.04


Средняя зарплата по специальностям найдена. В анализе я не заполнял пропуски зарплат. Хотя это можно было сделать с помощью заполнения средней зарплатой в группе (например, сгрупировать по специальности и городу, найти среднюю зарплату для этой группы и заполнить недостающие данные).

P.s. после итераций в разные дни количество вакансий в дате может менятся и появлятся новые валюты. Если такие появляются то код их не будет ичитывать и просто запишет в среднюю None. Ниже код для выявления аномалий (в первой строке поменять Specialization и вписать нужную сумму для проверки)

In [27]:
# filtered_df = analytical_df[(analytical_df['Specialization'] == 'Internet Marketer') & (analytical_df['Mean Salary'] >= 112000)]
# result_df = filtered_df.loc[:, ['Title', 'City', 'Salary', 'From Salary', 'To Salary', 'Mean Salary']]
# result_df.sort_values(by='Mean Salary', ascending=False, inplace=True)
# result_df.head(10)

<a name="1.1.3."></a>
### Часть 3 - Самые встречаемые би-граммы по каждой специальности
<font size="2">([к содержанию](#0.0))</font>

In [28]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
import re


# Загрузка ресурсов NLTK
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pavelspirin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
# Создание копии DataFrame
bigram_df = df.copy()

Напишем функцию для предобработки текста. Она будет осуществлять следующие операции:
- Удаление HTML тегов
- Приведение текста к нижнему регистру
- Удаление ненужных символов и пунктуации
- Токенизация текста

In [30]:
def preprocess_text(text):
    """
    Функция для предобработки текста.

    Параметры:
        - text (str): Входной текст для предобработки.

    Возвращает:
        list: Список токенов после предобработки текста.
        
    """
    text = re.sub(r'<[^>]+>', '', text)  # Удаление HTML тегов
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ\s]', '', text)  # Удаление ненужных символов, пунктуации
    tokens = nltk.word_tokenize(text)  # Токенизация текста
    
    return tokens

In [31]:
# Применение предобработки текста к столбцу Duties
bigram_df['Processed Duties'] = bigram_df['Duties'].apply(preprocess_text)

In [32]:
bigram_df.head()

,Title,Salary,Experience,Skills,Professional Role,Specialization,Duties,City,Link,Processed Duties
0,Frontend Developer (React),None,От 1 года до 3 лет,"[JavaScript, React, TypeScript, MobX, Bootstra...",None,Java Developer,"<p>Мы аккредитованная IT-компания, основанная ...",Екатеринбург,https://hh.ru/vacancy/94287939,"[мы, аккредитованная, itкомпания, основанная, ..."
1,HTML-верстальщик (HTML / CSS / JavaScript),"{'from': 70000, 'to': 100000, 'currency': 'RUR...",От 1 года до 3 лет,"[ReactJS, HTML, CSS, HTML5, JavaScript, Git, C...",None,SMM,<p>Привет!</p> <p>Мы - компания ISS.Digital. М...,Москва,https://hh.ru/vacancy/94318962,"[привет, мы, компания, issdigital, мы, создаем..."
2,Java программист,None,От 1 года до 3 лет,"[Java, PostgreSQL, Spring Boot, ORACLE]",None,Internet Marketer,<p><strong>ПРОЕКТ:</strong></p> <p>Ищем разраб...,Воронеж,https://hh.ru/vacancy/94268107,"[проект, ищем, разработчиков, java, для, разви..."
3,Ведущий программист Java (удалённо),None,От 3 до 6 лет,"[DevOps (Git (bitbucket), Jenkins, ansible, ma...",None,Web Designer,<p><em><strong>КОМПАНИЯ ООО ИЦ «АЙ-ТЕКО»</stro...,Москва,https://hh.ru/vacancy/92430462,"[компания, ооо, иц, айтеко, ведущий, российски..."
4,Младший разработчик,"{'from': 60000, 'to': None, 'currency': 'RUR',...",Нет опыта,[],None,Java Developer,<p>Вакансия для начинающих разработчиков с неб...,Санкт-Петербург,https://hh.ru/vacancy/94254962,"[вакансия, для, начинающих, разработчиков, с, ..."


In [33]:
def find_bigrams(tokens):
    """
    Функция для нахождения би-грамм в списке токенов

    Параметры:
        - tokens (list): Список токенов, в котором нужно найти би-граммы.

    Возвращает:
        list: Список би-грамм, найденных в заданных токенах.
    
    """
    bigrams = list(ngrams(tokens, 2))
    return bigrams

Создадим словарь `specialization_bigrams`, где ключами являются различные специализации из столбца `Specialization`, а значениями - би-граммы, найденные в текстовых данных признака `Processed Duties` для каждой специализации.

In [34]:
# Создание словаря для хранения би-грамм для каждой специализации
specialization_bigrams = {}

# Нахождение би-грамм для каждой специализации
for specialization in bigram_df['Specialization'].unique():
    specialization_tokens = bigram_df[bigram_df['Specialization'] == specialization]['Processed Duties'].sum()
    specialization_bigrams[specialization] = find_bigrams(specialization_tokens)

Выводем 10 самых часто встречающихся би-грамм для каждой специализации из словаря specialization_bigrams. Пройдемся по каждому элементу словаря и выводем название специализации и наиболее часто встречающиеся би-граммы для этой специализации вместе с их частотой.

In [35]:
# Нахождение 10 самых частых би-грамм для каждой специализации
for specialization, bigrams in specialization_bigrams.items():
    print(f'Специальностям: {specialization}')
    for bigram, count in Counter(bigrams).most_common(10):
        print(f'{bigram}: {count}')
    print('\n')

Специальностям: Java Developer
('опыт', 'работы'): 56
('работы', 'с'): 56
('участие', 'в'): 26
('умение', 'работать'): 19
('работать', 'с'): 16
('по', 'тк'): 14
('в', 'команде'): 13
('работы', 'в'): 12
('тк', 'рф'): 11
('оформление', 'по'): 11


Специальностям: SMM
('опыт', 'работы'): 61
('работы', 'с'): 49
('участие', 'в'): 26
('работа', 'в'): 20
('работы', 'в'): 16
('в', 'команде'): 15
('с', 'до'): 14
('что', 'мы'): 13
('тк', 'рф'): 13
('по', 'тк'): 11


Специальностям: Internet Marketer
('опыт', 'работы'): 43
('работы', 'с'): 37
('участие', 'в'): 16
('опыт', 'разработки'): 16
('у', 'нас'): 14
('тк', 'рф'): 14
('работы', 'в'): 11
('формат', 'работы'): 10
('оформление', 'по'): 10
('с', 'до'): 10


Специальностям: Web Designer
('опыт', 'работы'): 49
('работы', 'с'): 44
('мы', 'предлагаем'): 18
('график', 'работы'): 15
('тк', 'рф'): 13
('работы', 'в'): 13
('у', 'нас'): 13
('умение', 'работать'): 11
('работа', 'с'): 11
('участие', 'в'): 11




<a name="2.1."></a>
## Задание 2
<font size="2">([к содержанию](#0.0))</font>

Задача состоит в том, чтобы создать датафрейм, содержащий только те вакансии, которые присутствуют в первом файле, но отсутствуют во втором.

In [36]:
import pandas as pd

In [37]:
try:
    data_1 = pd.read_excel('internet_marketologs_part_1.xlsx')
    data_2 = pd.read_excel('internet_marketologs_part_2.xlsx')
    print('File load: Success')
except:
    print('File load: Failed')

File load: Success


In [38]:
data_1.head(2)

,name,skills,experiences,employments,professional_roles,city,salary_from,salary_to,currency,gross,schedules,description,url
0,Помощник интернет-маркетолога,"интернет-реклама, seo, яндекс.директ, таргетин...",Нет опыта,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Санкт-Петербург,50000,80000,RUR,False,Полный день,В маркетинговое агентство требуется Интернетма...,https://hh.ru/vacancy/89674701
1,Интернет-маркетолог (стажер),"продвижение бренда, планирование рекламных кам...",Нет опыта,Стажировка,"Менеджер по маркетингу, интернет-маркетолог",Новосибирск,30000,40000,RUR,False,Полный день,"Маркетика маркетинговое агентство, делаем инт...",https://hh.ru/vacancy/89023150


In [39]:
data_2.head(2)

,name,skills,experiences,employments,professional_roles,city,salary_from,salary_to,currency,gross,schedules,description,url
0,Интернет-маркетолог (Стажировка),"маркетинговые коммуникации, crm, управление пр...",Нет опыта,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Киров (Кировская область),40000,60000,RUR,False,Удаленная работа,"Интернетмаркетолог (Обучение) Привет, меня зов...",https://hh.ru/vacancy/89422910
1,Менеджер по работе с блогерами/Интернет-маркет...,"интернет-реклама, продвижение бренда, обучение...",Нет опыта,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Новосибирск,35000,70000,RUR,False,Полный день,Кто мы Brand. Ad международное рекламное аген...,https://hh.ru/vacancy/89255478


Единственный уникальный признак в обоих датасетах это `url`. Его и будем использовать для работы

In [40]:
# Создание датафрейма с вакансиями, присутствующими в первом датасете, но отсутствующими во втором
unique_vacancies_df = data_1[~data_1['url'].isin(data_2['url'])]

In [41]:
unique_vacancies_df

,name,skills,experiences,employments,professional_roles,city,salary_from,salary_to,currency,gross,schedules,description,url
3,Интернет-маркетолог,не указано,Нет опыта,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Самара,100000,не указано,RUR,False,Полный день,Обязанности Лидогенерация Контроль подрядчиков...,https://hh.ru/vacancy/89134967
5,Помощник руководителя (маркетолога)/Ученик/Инт...,не указано,Нет опыта,Стажировка,"Менеджер по маркетингу, интернет-маркетолог",Екатеринбург,50000,80000,RUR,False,Полный день,"В компанию quotМалиновкаquot, крупнейшему прои...",https://hh.ru/vacancy/88585477
6,Junior интернет-маркетолог,"яндекс.метрика, анализ конкурентной среды, ана...",Нет опыта,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Минск,800,2000,BYR,True,Удаленная работа,"Вакансия открыта до 1 декабря включительно Мы,...",https://hh.ru/vacancy/88648716
14,Маркетолог / интернет-маркетолог,не указано,Нет опыта,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Минск,1000,2000,BYR,False,Полный день,ОООВосходЭнерго на рынке электротехники более ...,https://hh.ru/vacancy/88514712
27,Интернет-маркетолог,не указано,Нет опыта,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Новосибирск,40000,не указано,RUR,False,Полный день,Кто мы и что делаем Доставляем грузы на самолё...,https://hh.ru/vacancy/89591853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,Интернет маркетолог,"маркетинговый анализ, google analytics, intern...",От 3 до 6 лет,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Москва,не указано,120000,RUR,False,Полный день,"Всем привет, международная производственная ко...",https://hh.ru/vacancy/88553608
1023,Интернет-маркетолог,"digital marketing, реклама, internet marketing...",От 3 до 6 лет,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Екатеринбург,не указано,не указано,не указано,False,Полный день,ООО quotЦентр косметологии и пластической хиру...,https://hh.ru/vacancy/89258682
1024,Интернет-маркетолог (Специалист по рекламе),"b2b маркетинг, интернет маркетинг, аналитическ...",От 3 до 6 лет,Полная занятость,"Менеджер по маркетингу, интернет-маркетолог",Чебоксары,55000,не указано,RUR,True,Полный день,"Мы группа компаний Точка Роста, производим обо...",https://hh.ru/vacancy/86258869
1026,"SMM - менеджер, интернет-маркетолог (работа в ...","smm, ведение групп в социальных сетях, продвиж...",От 3 до 6 лет,Полная занятость,"SMM-менеджер, контент-менеджер",Москва,80000,не указано,RUR,False,Полный день,Burevestnik Group ведущий яхтенный холдинг РФ...,https://hh.ru/vacancy/88651208


<a name="3.1."></a>
## Задание 3
<font size="2">([к содержанию](#0.0))</font>

Для построения простой рекомендательной системы будем использовать два признака `Skills` и `Duties`.  
**Предобработка:**  
- Уберем стоп-слова;
- Приведем все в нижний регистр;
- Леммитизация
- Токенизация

**Векторизация данных:**  
Чтобы использовать оба признака для рекомендации нужно скомбинировать их информацию, чтобы получить полный контекст каждой вакансии. Векторизацию будем выполнять при помощи метода TF-IDF.

**Векторизация запроса пользователя:**  
Применим обученый трансформер на запрос пользователя.

**Метрики растояния:**  
Будем измерять косинусное растояние между векторами в пространстве. Косинусное расстояние измеряет сходство направлений векторов и принимает значения от 0 до 1, где 0 означает идеальное совпадение, а 1 - полное несовпадение.

In [42]:
!pip install -q pymorphy2

In [43]:
# Импорт библиотек для обработки текста
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Загрузка ресурсов для NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Импорт библиотек для работы с данными
import pandas as pd
import ast

# Импорт библиотек для векторизации текста и вычисления косинусного сходства
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Импорт библиотеки для лемматизации слов
import pymorphy2

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pavelspirin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pavelspirin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pavelspirin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
print(df.Skills[0])

['JavaScript', 'React', 'TypeScript', 'MobX', 'Bootstrap', 'Git', 'Linux', 'Docker']


In [45]:
print(df.Duties[0])

<p>Мы аккредитованная IT-компания, основанная в 2015 году<strong> </strong>как одно из R&amp;D-подразделений группы компаний <strong>S&amp;T Iskratel</strong> (дочерняя структура группы <strong>Kontron AG</strong>, Австрия). Основные направления нашей деятельности - это продукты и решения по цифровизации и автоматизации производственных и управленческих процессов в таких отраслях как транспорт, энергетика, общественная безопасность и телекоммуникации.</p> <p>Одним из новых направлений работы для нас является проект по разработке ПО ядра современных сетей связи на базе технологий 4G/5G. Данный проект предполагает реализацию решения, обеспечивающего необходимые функции мобильных сетей четвертого и пятого поколения в соответствии со спецификациями комитета по стандартизации технологий мобильной связи 3GPP. Большой объем работы по этим проектам предполагает расширение команды, поэтому мы <strong>ищем опытного Frontend разработчика (React)</strong>.</p> <p><strong>Наш стек: </strong>Typescr

In [46]:
# Объединение, используя метод .str.join() и обычную конкатенацию строк
df['Combined_text'] = df['Skills'].str.join(' ').astype(str) + ' ' + df['Duties'].str.strip()

In [47]:
df.Combined_text[0]

'JavaScript React TypeScript MobX Bootstrap Git Linux Docker <p>Мы аккредитованная IT-компания, основанная в 2015 году<strong> </strong>как одно из R&amp;D-подразделений группы компаний <strong>S&amp;T Iskratel</strong> (дочерняя структура группы <strong>Kontron AG</strong>, Австрия). Основные направления нашей деятельности - это продукты и решения по цифровизации и автоматизации производственных и управленческих процессов в таких отраслях как транспорт, энергетика, общественная безопасность и телекоммуникации.</p> <p>Одним из новых направлений работы для нас является проект по разработке ПО ядра современных сетей связи на базе технологий 4G/5G. Данный проект предполагает реализацию решения, обеспечивающего необходимые функции мобильных сетей четвертого и пятого поколения в соответствии со спецификациями комитета по стандартизации технологий мобильной связи 3GPP. Большой объем работы по этим проектам предполагает расширение команды, поэтому мы <strong>ищем опытного Frontend разработчик

Возьмем стоп-слова для двух языков, так как в признаках у нас есть как русские, так и английские слова. Так же добавим свои стоп-слова (после множественных итераций выявил некторые символы, которые не удаляется), а затем объеденим

In [48]:
# Стоп-слова для английского и русского языков из NLTK
english_stop_words = set(stopwords.words('english'))
russian_stop_words = set(stopwords.words('russian'))

# Дополнительные стоп-слова
additional_stop_words = {'(', ')', '/', ':', ';'}

# Объединение стоп-слов для обоих языков и дополнительных слов
stop_words = english_stop_words.union(russian_stop_words, additional_stop_words)

Далее напишем функцию по предобработке данных. Будем использовать лемматизатор для русского языка из библиотеки pymorphy2. 

In [49]:
# Создание экземпляра лемматизатора
morph = pymorphy2.MorphAnalyzer()

In [50]:
# Функция для предобработки текста
def preprocess_text(text):
    # Проверка, является ли text строкой и не равен None
    if isinstance(text, str) and text is not None:
        text = re.sub(r'<[^>]+>', '', text)  # Удаление HTML тегов
        text = text.lower()  # Приведение к нижнему регистру
        tokens = word_tokenize(text)  # Токенизация текста
        lemmatized_tokens = [morph.parse(word)[0].normal_form for word in tokens]  # Лемматизация слов
        filtered_tokens = [word for word in lemmatized_tokens if word not in stop_words]  # Удаление стоп-слов
        return ' '.join(filtered_tokens)  # Возврат строки
    else:
        # Если text не является строкой или равен None, вернуть его без изменений
        return text

In [51]:
# Применение предобработки
df['Combined_text'] = df['Combined_text'].apply(preprocess_text)

In [52]:
print(df.Combined_text[0])

javascript react typescript mobx bootstrap git linux docker аккредитовать it-компания , основать 2015 год r & amp d-подразделение группа компания & amp iskratel дочерний структура группа kontron ag , австрия . основной направление наш деятельность - это продукт решение цифровизация автоматизация производственный управленческий процесс отрасль транспорт , энергетика , общественный безопасность телекоммуникация . новый направление работа являться проект разработка ядро современный сеть связь база технология 4g/5g . данный проект предполагать реализация решение , обеспечивать необходимый функция мобильный сеть четвёртый пятый поколение соответствие спецификация комитет стандартизация технология мобильный связь 3gpp . большой объём работа это проект предполагать расширение команда , поэтому искать опытный frontend разработчик react . наш стек typescript 4+ , react 17+ , bootstrap 4 , redux/mobx , postgresql , redis , docker , jenkins , linux ubuntu 20.04 lts , java/spring backend , angular

Видим, что предобработка прошла успешно. Далее необходимо векторизировать токены. Будем использовать метод TF-IDF

In [53]:
# Создание векторизатора TF-IDF
tfidf = TfidfVectorizer()

# Преобразование текста в признаки TF-IDF
tfidf_matrix = tfidf.fit_transform(df['Combined_text'])

In [54]:
tfidf_matrix

<146x4759 sparse matrix of type '<class 'numpy.float64'>'
	with 20891 stored elements in Compressed Sparse Row format>

Матрица призаков корпуса готова. Далее имитируем запрос пользователя, применим к нему функцию предобработки текста и векторизируем предобученым трансформером.

In [55]:
# Пример запроса пользователя
user_query = 'Я програмист java, стажер, знаю Python, ООП, REST, Kotlin, DevOps, Git, bitbucket, mysql, хочу работать на удаленке, мотивирован, немного знаю NLP'
# user_query = 'Java Spring Framework Kotlin Kafka REST Знание CI/CD'
# user_query = 'SMM Junior'

# Предобработка запроса
preprocessed_query = preprocess_text(user_query)

# Векторизация запроса
query_vector = tfidf.transform([preprocessed_query])

Далее измерим косинусное растояние между векторами в пространстве.

In [56]:
# Вычисляем косинусное сходство между запросом и каждой вакансией и получаем матрицу
similarities = cosine_similarity(query_vector, tfidf_matrix)

In [57]:
# Выводим результат топ-N рекомендуемых вакансий
print('Топ-10 рекомендуемых вакансий:')
print()

for index in similarities.argsort()[0][::-1][:10]:
    print(f"Название: {df.iloc[index]['Title']}")
    print(f"Ссылка: {df.iloc[index]['Link']}")
    print()

Топ-10 рекомендуемых вакансий:

Название: Ведущий программист Java (удалённо)
Ссылка: https://hh.ru/vacancy/92430462

Название: Android-разработчик (удаленно)
Ссылка: https://hh.ru/vacancy/92754465

Название: Python разработчик / Ко-фаундер (Удалено)
Ссылка: https://hh.ru/vacancy/94320025

Название: Программист-разработчик
Ссылка: https://hh.ru/vacancy/94123275

Название: Стажер-разработчик программного обеспечения
Ссылка: https://hh.ru/vacancy/94259442

Название: Backend разработчик
Ссылка: https://hh.ru/vacancy/93429262

Название: Разработчик Android
Ссылка: https://hh.ru/vacancy/94098479

Название: Web-программист - стажер
Ссылка: https://hh.ru/vacancy/93597402

Название: Веб разработчик Full Stack, Front / Back
Ссылка: https://hh.ru/vacancy/94033474

Название: Программист-разработчик (стажер)
Ссылка: https://hh.ru/vacancy/94281477

